# Data preparation for UHI modeling

In [87]:
# start-up
import pickle
import yaml
config_path = '/home/tu/tu_tu/tu_zxmny46/DS_Project/modules/config.yml'

with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [88]:
# import additional packages
import math
import folium
import numpy as np
import geopandas as gpd
from IPython.display import display
from shapely.geometry import Polygon

In [89]:
def convert_lon_lat(point):
    return [point[1], point[0]]

def convert_lon_lat_bbox(bbox):
    converted_bbox = [convert_lon_lat(bbox[:2])]
    converted_bbox.append(convert_lon_lat(bbox[2:]))
    return [coord for point in converted_bbox for coord in point]

In [90]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    # Convert latitude and longitude to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate the differences between the latitudes and longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Apply the haversine formula
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000  # Convert distance to meters

    return distance

In [91]:
def adjust_coordinates(coordinates, mult=100):
    lat1, lon1, lat2, lon2 = coordinates

    # Calculate the distance between the longitude points
    lon_distance = haversine(lat1, lon1, lat1, lon2)

    # Calculate the distance between the latitude points
    lat_distance = haversine(lat1, lon1, lat2, lon1)

    # Calculate the adjusted longitude values
    lon_adjustment = math.ceil(lon_distance / mult) * mult - lon_distance
    new_lon1 = lon1 - lon_adjustment / (111320 * math.cos(math.radians(lat1)))  # Convert meters to degrees
    new_lon2 = lon2 + lon_adjustment / (111320 * math.cos(math.radians(lat2)))  # Convert meters to degrees

    # Calculate the adjusted latitude values
    lat_adjustment = math.ceil(lat_distance / mult) * mult - lat_distance
    new_lat1 = lat1 - lat_adjustment / 111320  # Convert meters to degrees
    new_lat2 = lat2 + lat_adjustment / 111320  # Convert meters to degrees

    return [new_lat1, new_lon1, new_lat2, new_lon2]

In [92]:
def divide_into_grid(coordinates, mult):
    lat1, lon1, lat2, lon2 = coordinates

    # Calculate the number of rows and columns in the grid
    lat_distance = haversine(lat1, lon1, lat2, lon1)
    lon_distance = haversine(lat1, lon1, lat1, lon2)
    num_rows = int(lat_distance / mult)
    num_cols = int(lon_distance / mult)

    # Calculate the size of each grid cell in degrees
    lat_cell_size = (lat2 - lat1) / num_rows
    lon_cell_size = (lon2 - lon1) / num_cols

    # Initialize the grid to store the individual areas
    grid = []

    # Iterate over the rows and columns to create the individual areas
    for row in range(num_rows):
        for col in range(num_cols):
            # Calculate the coordinates of the current grid cell
            cell_lat1 = lat1 + row * lat_cell_size
            cell_lon1 = lon1 + col * lon_cell_size
            cell_lat2 = cell_lat1 + lat_cell_size
            cell_lon2 = cell_lon1 + lon_cell_size

            # Add the coordinates of the current grid cell to the grid
            grid.append([cell_lat1, cell_lon1, cell_lat2, cell_lon2])

    return grid

In [93]:
coordinates = config['bboxes']['munich']
print(coordinates)
mult = 5000
adjusted_coordinates = adjust_coordinates(coordinates=convert_lon_lat_bbox(coordinates), mult=mult)
print(adjusted_coordinates)
grid = divide_into_grid(adjusted_coordinates, mult)

[11.31079, 47.980475, 11.855986, 48.385189]
[47.935578696928005, 11.251473590956458, 48.43008530307199, 11.915772596018419]


In [94]:
print(grid[0])

[47.935578696928005, 11.251473590956458, 47.9850293575424, 11.325284591518898]


In [95]:
def visualize_grid(coordinates, grid):
    # Calculate the center of the bounding box
    lat_center = (coordinates[0] + coordinates[2]) / 2
    lon_center = (coordinates[1] + coordinates[3]) / 2


    # Create a Folium map centered at the bounding box center
    map_center = [lat_center, lon_center]
    m = folium.Map(location=map_center, zoom_start=12)

    # Add the grid cells as rectangles to the map
    for cell in grid:
        folium.Rectangle(bounds=[[cell[0], cell[1]], [cell[2], cell[3]]],
                         color='black',
                         fill=False,
                         opacity=0.1).add_to(m)
    
    # Add the bounding box as a rectangle to the map
    folium.Rectangle(bounds=[[coordinates[0], coordinates[1]], [coordinates[2], coordinates[3]]],
                     color='red',
                     fill=False).add_to(m)
    
    # Display the map in the Jupyter Notebook
    display(m)

In [ ]:
visualize_grid(adjusted_coordinates, grid)

In [97]:
def aggregate_temperature_data(data, grid):
    # Create an empty array to store the aggregated temperature values
    aggregated_data = np.zeros(len(grid))

    # Iterate over each grid cell
    for i, cell in enumerate(grid):
        # Extract the coordinates of the grid cell
        lon1, lat1, lon2, lat2 = cell

        # Create a Polygon object for the grid cell
        cell_polygon = Polygon([(lon1, lat1), (lon2, lat1), (lon2, lat2), (lon1, lat2)])

        # Calculate the area of the grid cell
        cell_area = cell_polygon.area

        # Get the corresponding land surface temperature data within the grid cell
        cell_data = data.cx[lon1:lon2, lat1:lat2]

        # Calculate the weighted mean temperature for the grid cell
        weighted_mean = np.average(cell_data, weights=cell_area)

        # Store the aggregated temperature value for the grid cell
        aggregated_data[i] = weighted_mean

    return aggregated_data

In [98]:
# aggregated_data = aggregate_temperature_data(data, grid)

In [99]:
# data = {'X': X, 'Y': Y}
# with open(config['data'] + '/uhi_model/uhi_data.pkl', 'wb') as file:
#     pickle.dump(aggregated_data, file)